# Covid Chest Project

## Setup

In [19]:
import numpy as np
from utils import load_covid_data, create_dataset_xray
from models import fusion_model
from plots import plot_cm_handy, plot_roc_handy
from evaluation import mode_robustness, evaluation
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import load_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## Loading Data

In [ ]:
image_size = 150
x_train, y_train = load_covid_data(image_size, path='Data/train',shuffle=True, class_frequency=True)
x_test,y_test = load_covid_data(image_size, path='Data/test')

class_weights = compute_class_weight('balanced',np.unique(np.argmax(y_train,axis=1)), np.argmax(y_train,axis=1))
class_weights={0:class_weights[0],
               1:class_weights[1],
               2:class_weights[2]}


train_dataset,validation_dataset=create_dataset_xray(x_train, y_train, x_test, y_test,batch_size=128)

## Defining Model

In [ ]:
mc_model, mc_callbacks = fusion_model(mc=True)
simple_model, simple_callbacks = fusion_model(mc=False)

## Training Model

In [ ]:
hist_mc = mc_model.fit(train_dataset, epochs=200, validation_data=validation_dataset,
                          class_weight=class_weights, callbacks=mc_callbacks)

## Noise Addition & Robustness Quantification

In [ ]:
mc_model=load_model('Chest_Covid/model_covid_mc.h5')

mean_coef=[1,1,1,1,1,1,1,1]
std_coef=[1e-4,1e-3,1e-2,1e-1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,
          1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]
result=mode_robustness(x_test,y_test,mc_model,std_coef)
print(result)

## Plot Roc

In [ ]:
y_p = mc_model.predict(x_test,batch_size=BATCH_SIZE)

plot_roc_handy(y_test,y_pred ,zoom=True,lw=2,name='ROC of Random Forest (X-Ray)',
               class_name=['COVID19','Normal','Pneumonia'])

## Plot Confusion Matrix

In [ ]:
plot_cm_handy(y_test,y_p,
              lw=2,name='Confusion Matrix of Decision Tree (X-Ray)',
              class_name=['COVID19','Normal','Pneumonia'])

## Statistical Methods

In [ ]:
x_train_statistical=x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2]*x_train.shape[3])
y_train_statistical=np.argmax(y_train,axis=1)
x_test_statistical=x_test.reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]*x_test.shape[3])
y_test_statistical=np.argmax(y_test,axis=1)

classifier1=DecisionTreeClassifier(max_depth=50,class_weight='balanced')
param_grid_dt = {'max_depth':[50,100,150,200,300]}
grid = GridSearchCV(DecisionTreeClassifier(class_weight='balanced',random_state=0) ,param_grid_dt,cv=3)
classifier1.fit(x_train_statistical, y_train_statistical)
print('Best params of dt is:',grid.get_params)
classifier1=grid.best_estimator_
y_pred_dt=classifier1.predict(x_test_statistical)
acc,precision,recall,F1=evaluation(y_test_statistical,y_pred_dt)
print('Dt test results are:',evaluation(y_test_statistical,y_pred_dt))
print('Dt train results are:',evaluation(y_train_statistical,classifier1.predict(x_train_statistical)))

In [ ]:
classifier2=RandomForestClassifier(max_depth=50,n_estimators=200,class_weight='balanced',random_state=0)
classifier2.fit(x_train_statistical, y_train_statistical)

y_pred_rf=classifier2.predict(x_test_statistical)
acc,precision,recall,F1=evaluation(y_test_statistical,y_pred_rf)
print('Rf test results are:',evaluation(y_test_statistical,y_pred_rf))
print('Rf train results are:',evaluation(y_train_statistical,classifier1.predict(x_train_statistical)))



In [ ]:
from sklearn.metrics import plot_roc_curve, roc_curve, auc
from scikitplot.metrics import plot_confusion_matrix,plot_roc
from scipy import interp
from itertools import cycle

plot_roc_handy(y_test, y_p,lw=2,name='Roc of fusion model without uncertainty (X-Ray)',
               class_name=['COVID19','Normal','Pneumonia'])

## Ensemble MCD